This notebook lets you download a list of songs from youtube, performs audio source separation to extract the voices from the songs and if desired, removes all the silences and adds the snippets together to one long audio file. The output is automatically saved in your google drive in a folder with the name defined at "folder_name"

In [ ]:
#mount your google drive to save the output to
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
folder_name="jack_johnson_yt/" #where to save to in drive

#all eddie vedder songs from into the wild except for "tuolumne" because thats only instrumental
'''
youtube_list=[
"https://www.youtube.com/watch?v=KDlsH8MbDbs",
"https://www.youtube.com/watch?v=oO4O8Xpvp_Q",
"https://www.youtube.com/watch?v=zP-6sau1Jmo",
"https://www.youtube.com/watch?v=GXQSRuLeSEM",
"https://www.youtube.com/watch?v=S5LFCE-_hNY",
"https://www.youtube.com/watch?v=_EZUfnMv3Lg",
"https://www.youtube.com/watch?v=ABsslEoL0-c",
"https://www.youtube.com/watch?v=CB4xAL1rySw",
"https://www.youtube.com/watch?v=hqaeeQkE2dE",
"https://www.youtube.com/watch?v=K6g4841NZ6I",
]
'''


#jack johnson album "Between Dreams"

youtube_list=[
"https://youtu.be/P0rKYkpcnB0?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/AtkBgbQxBs0?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/1gQvaLsQi3Q?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/cG8kBsjfVp8?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/b8vl_-yXNDM?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/nY9qR3EpRDE?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/5XTon19ueJY?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/YMc0sRjmKqk?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/k1WRjejBjmk?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/9AZVXorIZNs?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/zwi4nJjS9zk?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/CZbORKIAyOU?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/N_ugWaw-fTo?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
"https://youtu.be/6g4oyd-h9z8?list=PLH2nYalExXL0GFXWfk0iCCcjrpHLI4Wnh",
]


In [ ]:
#install pytube to download youtube songs
!pip install git+https://gitlab.com/obuilds/public/pytube@ob-v1  --upgrade

#install spleeter to separate audio sources
!apt install ffmpeg
!pip install spleeter

In [ ]:
from pytube import YouTube
from IPython.display import Audio
import librosa
import numpy as np
import soundfile as sf

In [ ]:
%cd /content
!rm -r downloads
!mkdir downloads

In [ ]:
%cd /content/downloads
mp4_files = []
for i,track in enumerate(youtube_list):
  yt = YouTube(track)
  mp4_stream=yt.streams.get_by_itag('140')
  mp4_stream.download(filename=str(i))
  mp4_files.append(str(i)+".mp4")

In [ ]:
%cd /content/downloads
for filename in mp4_files:
  print(filename)
  !spleeter separate -i  $filename -p spleeter:2stems -o output
  
#for 16khz model instead of 11khz: (aparently gives artefacts)
#!spleeter separate -i audio_example.mp3 -o audio_output -p spleeter:4stems-16kHz

In [ ]:
#folder name defined above (first cell)
!rm -r output_vocals
!mkdir output_vocals
long_vocals=[]
for filename in mp4_files:
  folder_number=filename[:-4]
  vocal_path="/content/downloads/output/"+folder_number+"/vocals.wav"
  vocal_target="/content/downloads/output_vocals/"+folder_number+".wav"
  !cp $vocal_path $vocal_target 
  long_vocals.append(vocal_target)

In [ ]:
print(long_vocals)

In [ ]:
#remove the silence from a song and save to drive

HOME_DIR_LINUX="/content/drive/My\ Drive/" #because its a huge pain that "My Drive" has a f*ing empty space in the name
HOME_DIR_LIBROSA="/content/drive/My Drive/"
target_linux=HOME_DIR_LINUX+folder_name
target_librosa=HOME_DIR_LIBROSA+folder_name
!mkdir $target_linux
for i,v in enumerate(long_vocals):
  audio,sr=librosa.core.load(v,16000,mono=True)
  splits=librosa.effects.split(y=audio, frame_length=sr, top_db=20)
  short_audio=np.array([])
  for j,split in enumerate(splits):
    short_audio=np.append(short_audio,audio[split[0]:split[1]])
  sf.write(target_librosa+str(i)+".wav", short_audio, sr)
  #librosa.output.write_wav(target_librosa+str(i)+".wav",short_audio, sr)

In [ ]:
#audio,sr=librosa.core.load("/content/downloads/output_vocals/8.wav",16000,mono=True)
Audio(short_audio,rate=sr)
#librosa.output.write_wav(target_librosa+"test.wav",short_audio, sr)

In [ ]:
#add them all together to one file:
import wave
import glob
import os
%cd $target_linux

wav_files = glob.glob( '*.wav')
print (wav_files)

outfile=wav_files[0]
for i,file in enumerate(wav_files):
  if(i!=0):  
    infiles = [outfile,file]
    outfile = "alltogether.wav"

    data= []
    for infile in infiles:
        w = wave.open(infile, 'rb')
        data.append( [w.getparams(), w.readframes(w.getnframes())] )
        w.close()
        
    output = wave.open(outfile, 'wb')
    output.setparams(data[0][0])
    output.writeframes(data[0][1])
    output.writeframes(data[1][1])
    output.close()

%cd /content/